### In this jupyter notebook, we will practice reading catalog in FITS file, make sample selections, and take a look at the galaxies by making cropped images centered at them. In this exmple, we use catalog of galaxies detected in the NGDEEP field

In [1]:
%matplotlib inline

import os
import random
import numpy as np
import pandas as pd
import astropy.stats
from astropy.io import fits
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as fm
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] ='Times New Roman'

## (1) Read the galaxy catalog and get some basic parameters (e.g., coordinates, redshift, mass, and SFR)

In [2]:
# Directory of the catalog: where the catalog is located
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/All Catalogs/'

cat_eazy = fits.open(cat_dir+'allcats_zout.fits')
cat_eazy.info()
cat_eazy[1].header

Filename: D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/All Catalogs/allcats_zout.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (19767,)   uint8   
  1  /Users/itsnakhoirulfitriana/Documents/student_project_itb_galclum/ca    1 BinTableHDU    199   216098R x 83C   [K, 14A, D, D, D, K, E, E, E, D, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E]   


XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional table                            
NAXIS1  =                  978 / width of table in bytes                        
NAXIS2  =               216098 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group                                 
TFIELDS =                   83 / number of columns                              
EXTNAME = '/Users/itsnakhoirulfitriana/Documents/student_project_itb_galclum/ca'
TTYPE1  = 'index   '           / label for column 1                             
TFORM1  = 'K       '           / format for column 1                            
TCOMM1  = 'Expression: index'                                                   
TTYPE2  = 'ID_new  '        

In [3]:
# get the catalog data
cat_eazy_data = cat_eazy[1].data
cat_eazy_data

FITS_rec([(     1, 'CEERS1', 215.21529079, 52.97537283, -1., 0, -1.        , -1.        , -1.        , 4.31882810e+13,     0.        , -1.        , -1.        , -1.        , 0.01      , 0.        , 0.01      ,  0.       , 0.        , 0.        , 0.        , 0.       , 0.       , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ,     0.        , -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, [-9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29], [-9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29], [-9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29], [-9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.000

In [4]:
# Combined ID from all fields
ID_new = cat_eazy_data['ID_new']

# Right ascension and declination
ra = cat_eazy_data['ra']
dec = cat_eazy_data['dec']

# redshifts: spectroscopic and photometric redshifts
z_spec = cat_eazy_data['z_spec']
z_phot = cat_eazy_data['z_phot']

# rest-frame magnitudes in U, V, and J bands
restU = cat_eazy_data['restU']
restV = cat_eazy_data['restV']
restJ = cat_eazy_data['restJ']

# luminosity in U, V, and J bands
Lu = cat_eazy_data['Lu']
Lv = cat_eazy_data['Lv']
Lj = cat_eazy_data['Lj']

# stellar mass (SM) and star formation rate (SFR)
SM = cat_eazy_data['mass']
SFR = cat_eazy_data['sfr']
SM_p = cat_eazy_data['mass_p']
SM1 = np.array([x[1] for x in SM_p])
SM3 = np.array([x[3] for x in SM_p])
SFR_p = cat_eazy_data['sfr_p']
SFR1 = np.array([x[1] for x in SFR_p])
SFR3 = np.array([x[3] for x in SFR_p])

# number of objects (i.e., galaxies)
ngals = len(ra)
print ('Number of galaxies in the catalog: %d' % ngals)

cat_eazy.close()

Number of galaxies in the catalog: 216098


## (2) Read the photometry catalog

In [5]:
cat_photo = fits.open(cat_dir+'allcats_phot_apcorr.fits')
cat_photo.info()
cat_photo[1].header

Filename: D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/All Catalogs/allcats_phot_apcorr.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (95519,)   uint8   
  1  /Users/itsnakhoirulfitriana/Documents/student_project_itb_galclum/ca    1 BinTableHDU   2554   216098R x 890C   [K, 14A, D, K, K, K, K, K, K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, K, K, K, K, K, D, D, J, D, D, D, D, D, D, K, D, D, D, D, D, D, K, D, I, D, D, D, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional table                            
NAXIS1  =                 6576 / width of table in bytes                        
NAXIS2  =               216098 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group                                 
TFIELDS =                  890 / number of columns                              
EXTNAME = '/Users/itsnakhoirulfitriana/Documents/student_project_itb_galclum/ca'
TTYPE1  = 'index   '           / label for column 1                             
TFORM1  = 'K       '           / format for column 1                            
TCOMM1  = 'Expression: index'                                                   
TTYPE2  = 'ID_new  '        

In [6]:
# get the catalog data
cat_photo_data = cat_photo[1].data

rkron = cat_photo_data['kron_radius']
flux_iso = cat_photo_data['flux_iso']
flux_radius = cat_photo_data['flux_radius']
flux_radius20 = cat_photo_data['flux_radius_20']
flux_radius90 = cat_photo_data['flux_radius_90']

In [7]:
# check the list of filters and their names from the header information printed above
filter_names = ['f090w', 'f105w', 'f110w', 'f115w', 'f125w', 'f140w', 'f150w', 'f160w',
                'f182m', 'f200w', 'f210m', 'f277w', 'f335m', 'f356w', 'f410m', 'f430m',
                'f435w', 'f444w', 'f460m', 'f475w', 'f480m', 'f606w', 'f606wu', 'f775w',
                'f814w', 'f814wu', 'f850lp', 'f850lpu']
nfilters = len(filter_names)
print ('Number of filters: %d' % nfilters)

# get central wavelength of the filters
# filter_cwave = np.zeros(nfilters)
# for bb in range(nfilters):
#     filter_cwave[bb] = cat_photo[1].header[filter_names[bb]+'_PLAM']

Number of filters: 28


In [8]:
# get the fluxes in units of micro Jansky (uJy)
# The measurements were done with circular aperture with 0.7 arcsecond diameter
# Now the fluxes have been corrected for the flux loss due to the small aperture 
flux = {}
flux_err = {}
for bb in range(nfilters):
    flux[filter_names[bb]] = np.zeros(ngals)
    flux_err[filter_names[bb]] = np.zeros(ngals)
    
for ii in range(ngals):
    for bb in range(nfilters):
        flux[filter_names[bb]][ii] = cat_photo_data[filter_names[bb]+'_tot_2'][ii]
        flux_err[filter_names[bb]][ii] = cat_photo_data[filter_names[bb]+'_etot_2'][ii]

In [9]:
# calculate AB magnitudes
ABmag = {}
for bb in range(nfilters):
    ABmag[filter_names[bb]] = np.zeros(ngals)
    
for ii in range(ngals):
    for bb in range(nfilters):
        ABmag[filter_names[bb]][ii] = -2.5*np.log10(flux[filter_names[bb]][ii]*1e-6) + 8.90

cat_photo.close()

C:\Users\Ryo\AppData\Local\Temp\ipykernel_8680\2960262805.py:8: RuntimeWarning: invalid value encountered in log10
  ABmag[filter_names[bb]][ii] = -2.5*np.log10(flux[filter_names[bb]][ii]*1e-6) + 8.90


## Cutout Image

Pembuatan cutout image

size: 1.5"

filter: f115w, f150w, f277w, f444w

1. z>4.25 
rest frame H-alpha: f444w (penambahan kontur di imagenya)

2. 3<z<4.25
rest frame H-alpha: f277w (penambahan kontur di imagenya)

3. 2.25<z<3
rest frame H-alpha: f277w (penambahan kontur di imagenya)

In [10]:
idx_bin = np.where((z_spec>3) & (np.log10(SM)>7.0) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flux['f277w']>0) & (flux['f444w']>0))
#idx_bin = np.where((z_spec>1) & (z_spec<3) & (np.log10(SM)>8.5) & (np.log10(SM)<11.5) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flux['f277w']>0) & (flux['f444w']>0))
data = {'ID' : [ID_new[ii] for ii in idx_bin[0]], 
        'RA' : [ra[ii] for ii in idx_bin[0]],
        'Dec' : [dec[ii] for ii in idx_bin[0]],
        'z_sp' : [z_spec[ii] for ii in idx_bin[0]],
        'log_SM' : [np.log10(SM[ii]) for ii in idx_bin[0]],
        'log_SFR' : [np.log10(SFR[ii]) for ii in idx_bin[0]],
        'log_sSFR' : [np.log10(SFR[ii])-np.log10(SM[ii]) for ii in idx_bin[0]]}

df_sel = pd.DataFrame(data, index=[ii for ii in idx_bin[0]])
df_z = df_sel.sort_values(by=['z_sp'])
idx = df_z.index

C:\Users\Ryo\AppData\Local\Temp\ipykernel_8680\922003286.py:1: RuntimeWarning: divide by zero encountered in log10
  idx_bin = np.where((z_spec>3) & (np.log10(SM)>7.0) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flux['f277w']>0) & (flux['f444w']>0))
C:\Users\Ryo\AppData\Local\Temp\ipykernel_8680\922003286.py:1: RuntimeWarning: invalid value encountered in log10
  idx_bin = np.where((z_spec>3) & (np.log10(SM)>7.0) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flux['f277w']>0) & (flux['f444w']>0))
C:\Users\Ryo\AppData\Local\Temp\ipykernel_8680\922003286.py:1: RuntimeWarning: divide by zero encountered in divide
  idx_bin = np.where((z_spec>3) & (np.log10(SM)>7.0) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flux['f277w']>0) & (flux['f444w']>0))
C:\Users\Ryo\AppData\Local\Temp\ipykernel_8680\922003286.py:1: RuntimeWarnin

### CEERS

In [14]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/CEERS/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'ceers-full-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'ceers-full-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'ceers-full-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'ceers-full-grizli-v6.0-f444w-clear_drc_sci.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

{'f115w': 0.03999999999999923, 'f150w': 0.03999999999999923, 'f277w': 0.03999999999999923, 'f444w': 0.03999999999999923}


In [15]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch, MinMaxInterval, AsinhStretch, PercentileInterval
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.visualization import make_lupton_rgb
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

#matplotlib inline

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'CEERS58396' in x]

for ii in list:   
        
    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
        
    for bb in range(0,nfilters):

        dim_arcsec = 2  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

        stamp_size= [dim_pix,dim_pix] 
            
        hdu = fits.open(dir_images+img_name[filter_names[bb]])[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        hdu.data = cutout.data
        img_crop = hdu.data
        hdu.header.update(cutout.wcs.to_header())
        
        index = [3, 4, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = plt.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')

        level = np.linspace(img_crop.min()/2, img_crop.max(), 8)
        level = level[level != 0]
        cont = plt.contour(img_crop, levels=level, colors='gold', alpha=0.6)
        
        plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

        if bb == 0:
            b = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='blue', fontweight='bold')
            
        elif bb == 1: 
            g = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='green', fontweight='bold')

        elif bb == 2:
            r = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='red', fontweight='bold')

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='black', fontweight='bold')

    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())
    norm = ImageNormalize(interval=PercentileInterval(99.), stretch=AsinhStretch(a=0.85))
    r_norm = norm(r)
    g_norm = norm(g)
    b_norm = norm(b)
    rgb = plt.subplot(121, projection=wcs)
    
    rgb_default = make_lupton_rgb(r_norm, g_norm, b_norm, stretch=1.5, Q=0)
    plt.imshow(rgb_default, origin='lower')
    add_scalebar(rgb, scalebar_t, label='5 kpc', color='white', size_vertical=0.2, fontproperties=fm.FontProperties(weight='bold', size=15))
    plt.text(0.04, 0.92, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='left', verticalalignment='center',
                transform = rgb.transAxes, fontsize=25, color='white')
    #plt.text(0.8, 0.05, '%.1f\" = %.2f kpc' % (theta.value, distance_kpc.value),  horizontalalignment='center', verticalalignment='center', 
    #            transform = rgb.transAxes, fontsize=25, color='white', fontweight='bold')
    #plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
    lon = rgb.coords[0]
    lat = rgb.coords[1]
    
    lon.set_ticks_visible(False)
    lon.set_ticklabel_visible(False)
    lat.set_ticks_visible(False)
    lat.set_ticklabel_visible(False)
    lon.set_axislabel('')
    lat.set_axislabel('')

    name_out = 'z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

### COSWEB

In [ ]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)

# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/COSMOS-WEB/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'cosweb-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'cosweb-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'cosweb-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'cosweb-grizli-v6.0-f444w-clear_drc_sci.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'


# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.visualization import make_lupton_rgb
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

#%matplotlib inline

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'COSWEB23311' in x]

for ii in list:   
    
    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
        
    for bb in range(0,nfilters):

        dim_arcsec = 2  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

        stamp_size= [dim_pix,dim_pix] 
            
        hdu = fits.open(dir_images+img_name[filter_names[bb]])[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        hdu.data = cutout.data
        img_crop = hdu.data
        hdu.header.update(cutout.wcs.to_header())
        
        index = [3, 4, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = plt.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')

        level = np.linspace(img_crop.min()/2, img_crop.max(), 8)
        level = level[level != 0]
        cont = plt.contour(img_crop, levels=level, colors='gold', alpha=0.6)
        
        plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

        if bb == 0:
            b = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='blue', fontweight='bold')
            
        elif bb == 1: 
            g = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='green', fontweight='bold')

        elif bb == 2:
            r = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='red', fontweight='bold')

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='black', fontweight='bold')

    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    rgb = plt.subplot(121, projection=wcs)
    rgb_default = make_lupton_rgb(10*r, 10*g, 10*b)

    plt.imshow(rgb_default, origin='lower')
    add_scalebar(rgb, scalebar_t, label='5 kpc', color='white', size_vertical=0.2, fontproperties=fm.FontProperties(weight='bold', size=15))
    plt.text(0.04, 0.92, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='left', verticalalignment='center',
                transform = rgb.transAxes, fontsize=25, color='white')
    #plt.text(0.8, 0.05, '%.1f\" = %.2f kpc' % (theta.value, distance_kpc.value),  horizontalalignment='center', verticalalignment='center', 
    #            transform = rgb.transAxes, fontsize=25, color='white', fontweight='bold')
    #plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
    lon = rgb.coords[0]
    lat = rgb.coords[1]
    
    lon.set_ticks_visible(False)
    lon.set_ticklabel_visible(False)
    lat.set_ticks_visible(False)
    lat.set_ticklabel_visible(False)
    lon.set_axislabel('')
    lat.set_axislabel('')
    name_out = 'z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

### FRESCO

In [11]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f182m', 'f277w', 'f444w']

nfilters = len(filter_names)
               
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/FRESCO/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'gds-grizli-v6.1-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'gds-grizli-v6.1-f150w-clear_drc_sci.fits.gz'
img_name['f182m'] = 'gds-grizli-v6.1-f182m-clear_drc_sci.fits.gz'
img_name['f277w'] = 'gds-grizli-v6.1-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'gds-grizli-v6.1-f444w-clear_drc_sci.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f182m'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

{'f115w': 0.019999999999999896, 'f150w': 0.019999999999999896, 'f182m': 0.019999999999999896, 'f277w': 0.03999999999999973, 'f444w': 0.03999999999999973}


In [21]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch, MinMaxInterval, AsinhStretch, PercentileInterval
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.visualization import make_lupton_rgb
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

#%matplotlib inline

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'FRESCO14176' in x]

for ii in list:   
    
    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
        
    for bb in range(0,nfilters):

        dim_arcsec = 2  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

        stamp_size= [dim_pix,dim_pix] 
            
        hdu = fits.open(dir_images+img_name[filter_names[bb]])[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        hdu.data = cutout.data
        img_crop = hdu.data
        hdu.header.update(cutout.wcs.to_header())

        if bb == 2:
            r = img_crop
            continue

        index = [3, 4, 0, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = plt.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')

        level = np.linspace(img_crop.min()/2, img_crop.max(), 8)
        level = level[level != 0]
        cont = plt.contour(img_crop, levels=level, colors='gold', alpha=0.6)
        
        plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

        if bb == 0:
            b = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='blue', fontweight='bold')
            
        elif bb == 1: 
            g = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='green', fontweight='bold')

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='black', fontweight='bold')

        elif bb == 4:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='black', fontweight='bold')

    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    norm = ImageNormalize(interval=PercentileInterval(99.), stretch=AsinhStretch(a=0.85))
    r_norm = norm(r)
    g_norm = norm(g)
    b_norm = norm(b)
    rgb = plt.subplot(121, projection=wcs)
    
    rgb_default = make_lupton_rgb(r_norm, g_norm, b_norm, stretch=1.5, Q=0)
    plt.imshow(rgb_default, origin='lower')
    add_scalebar(rgb, scalebar_t, label='5 kpc', color='white', size_vertical=0.2, fontproperties=fm.FontProperties(weight='bold', size=15))
    plt.text(0.04, 0.92, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='left', verticalalignment='center',
                transform = rgb.transAxes, fontsize=25, color='white')
    #plt.text(0.8, 0.05, '%.1f\" = %.2f kpc' % (theta.value, distance_kpc.value),  horizontalalignment='center', verticalalignment='center', 
    #            transform = rgb.transAxes, fontsize=25, color='white', fontweight='bold')
    #plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
    lon = rgb.coords[0]
    lat = rgb.coords[1]
    
    lon.set_ticks_visible(False)
    lon.set_ticklabel_visible(False)
    lat.set_ticks_visible(False)
    lat.set_ticklabel_visible(False)
    lon.set_axislabel('')
    lat.set_axislabel('')
    name_out = 'z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

MemoryError: Unable to allocate 3.06 GiB for an array with shape (822083584,) and data type float32

In [34]:
np.where(r == 1)

(array([], dtype=int64), array([], dtype=int64))

### SED Cutout

In [ ]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
# filter_names = ['f090w', 'f105w', 'f110w', 'f115w', 'f125w', 'f140w', 'f150w', 'f160w',
#                'f182m', 'f200w', 'f210m', 'f277w', 'f335m', 'f356w', 'f410m', 'f430m',
#                'f435w', 'f444w', 'f460m', 'f475w', 'f480m', 'f606w', 'f606wu', 'f775w',
#                'f814w', 'f814wu', 'f850lp', 'f850lpu']
 
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/FRESCO/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f090w'] = 'gds-grizli-v6.1-f090w-clear_drc_sci.fits.gz'
img_name['f105w'] = 'gds-grizli-v6.1-f105w_drz_sci.fits.gz'
img_name['f110w'] = 'gds-grizli-v6.1-f110w_drz_sci.fits.gz'
img_name['f115w'] = 'gds-grizli-v6.1-f115w-clear_drc_sci.fits.gz'
img_name['f125w'] = 'gds-grizli-v6.1-f125w_drz_sci.fits.gz'
img_name['f140w'] = 'gds-grizli-v6.1-f140w_drz_sci.fits.gz'
img_name['f150w'] = 'gds-grizli-v6.1-f150w-clear_drc_sci.fits.gz'
img_name['f160w'] = 'gds-grizli-v6.1-f160w_drz_sci.fits.gz'
img_name['f182m'] = 'gds-grizli-v6.1-f182m-clear_drc_sci.fits.gz'
img_name['f200w'] = 'gds-grizli-v6.1-f200w-clear_drc_sci.fits.gz'
img_name['f210m'] = 'gds-grizli-v6.1-f210m-clear_drc_sci.fits.gz'
img_name['f277w'] = 'gds-grizli-v6.1-f277w-clear_drc_sci.fits.gz'
img_name['f335m'] = 'gds-grizli-v6.1-f335m-clear_drc_sci.fits.gz'
img_name['f356w'] = 'gds-grizli-v6.1-f356w-clear_drc_sci.fits.gz'
img_name['f410m'] = 'gds-grizli-v6.1-f410m-clear_drc_sci.fits.gz'
img_name['f430m'] = 'gds-grizli-v6.1-f430m-clear_drc_sci.fits.gz'
img_name['f435w'] = 'gds-grizli-v6.1-f435w_drc_sci.fits.gz'
img_name['f444w'] = 'gds-grizli-v6.1-f444w-clear_drc_sci.fits.gz'
img_name['f460m'] = 'gds-grizli-v6.1-f460m-clear_drc_sci.fits.gz'
img_name['f475w'] = 'gds-grizli-v6.1-f475w_drc_sci.fits.gz'
img_name['f480m'] = 'gds-grizli-v6.1-f480m-clear_drc_sci.fits.gz'
img_name['f606w'] = 'gds-grizli-v6.1-f606w_drc_sci.fits.gz'
img_name['f606wu'] = 'gds-grizli-v6.1-f606wu_drc_sci.fits.gz'
img_name['f775w'] = 'gds-grizli-v6.1-f775w_drc_sci.fits.gz'
img_name['f814w'] = 'gds-grizli-v6.1-f814w_drc_sci.fits.gz'
img_name['f814wu'] = 'gds-grizli-v6.1-f814wu_drc_sci.fits.gz'
img_name['f850lp'] = 'gds-grizli-v6.1-f850lp_drc_sci.fits.gz'
img_name['f850lpu'] = 'gds-grizli-v6.1-f850lpu_drc_sci.fits.gz'

telescope = {}
telescope['f090w'] = 'JWST'
telescope['f105w'] = 'HST'
telescope['f110w'] = 'HST'
telescope['f115w'] = 'JWST'
telescope['f125w'] = 'HST'
telescope['f140w'] = 'HST'
telescope['f150w'] = 'JWST'
telescope['f160w'] = 'HST'
telescope['f182m'] = 'JWST'
telescope['f200w'] = 'JWST'
telescope['f210m'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f335m'] = 'JWST'
telescope['f356w'] = 'JWST'
telescope['f410m'] = 'JWST'
telescope['f430m'] = 'JWST'
telescope['f435w'] = 'HST'
telescope['f444w'] = 'JWST'
telescope['f460m'] = 'JWST'
telescope['f475w'] = 'HST'
telescope['f480m'] = 'JWST'
telescope['f606w'] = 'HST'
telescope['f606wu'] = 'HST'
telescope['f775w'] = 'HST'
telescope['f814w'] = 'HST'
telescope['f814wu'] = 'HST'
telescope['f850lp'] = 'HST'
telescope['f850lpu'] = 'HST'


# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.visualization import make_lupton_rgb
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

#%matplotlib inline

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'FRESCO' in x]

for ii in list[0:1]:   
    
    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
        
    for bb in range(0,nfilters):

        dim_arcsec = 2  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

        stamp_size= [dim_pix,dim_pix] 
            
        hdu = fits.open(dir_images+img_name[filter_names[bb]])[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        hdu.data = cutout.data
        img_crop = hdu.data
        hdu.header.update(cutout.wcs.to_header())


        index = [3, 4, 0, 7, 8]
        ax = fig.add_subplot(4, 12, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = plt.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')

        level = np.linspace(img_crop.min()/2, img_crop.max(), 8)
        level = level[level != 0]
        cont = plt.contour(img_crop, levels=level, colors='gold', alpha=0.6)
        
        plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
        plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='black', fontweight='bold')

        if bb == 0:
            b = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='blue', fontweight='bold')
            
        elif bb == 1: 
            g = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='green', fontweight='bold')

        elif bb == 2:
            r = 

    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    rgb = plt.subplot(141, projection=wcs)
    rgb_default = make_lupton_rgb(100*r, 100*g, 100*b)

    plt.imshow(rgb_default, origin='lower')
    add_scalebar(rgb, scalebar_t, label='5 kpc', color='white', size_vertical=0.2, fontproperties=fm.FontProperties(weight='bold', size=15))
    plt.text(0.04, 0.92, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='left', verticalalignment='center',
                transform = rgb.transAxes, fontsize=25, color='white')
    #plt.text(0.8, 0.05, '%.1f\" = %.2f kpc' % (theta.value, distance_kpc.value),  horizontalalignment='center', verticalalignment='center', 
    #            transform = rgb.transAxes, fontsize=25, color='white', fontweight='bold')
    #plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
    lon = rgb.coords[0]
    lat = rgb.coords[1]
    
    lon.set_ticks_visible(False)
    lon.set_ticklabel_visible(False)
    lat.set_ticks_visible(False)
    lat.set_ticklabel_visible(False)
    lon.set_axislabel('')
    lat.set_axislabel('')
    name_out = 'z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

### PRIME_UDS

In [ ]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
               
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/PRIMER-UDS/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'primer-uds-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'primer-uds-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'primer-uds-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'primer-uds-grizli-v6.0-f444w-clear_drc_sci.fits.gz'


telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'


# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.visualization import make_lupton_rgb
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

#%matplotlib inline

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'PRIME_UDS' in x]

for ii in list[42:]:   
    
    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
        
    for bb in range(0,nfilters):

        dim_arcsec = 2  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

        stamp_size= [dim_pix,dim_pix] 
            
        hdu = fits.open(dir_images+img_name[filter_names[bb]])[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        hdu.data = cutout.data
        img_crop = hdu.data
        hdu.header.update(cutout.wcs.to_header())
        
        index = [3, 4, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = plt.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')

        level = np.linspace(img_crop.min()/2, img_crop.max(), 8)
        level = level[level != 0]
        cont = plt.contour(img_crop, levels=level, colors='gold', alpha=0.6)
        
        plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

        if bb == 0:
            b = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='blue', fontweight='bold')
            
        elif bb == 1: 
            g = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='green', fontweight='bold')

        elif bb == 2:
            r = img_crop
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='red', fontweight='bold')

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='black', fontweight='bold')

    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    rgb = plt.subplot(121, projection=wcs)
    rgb_default = make_lupton_rgb(10*r, 10*g, 10*b)

    plt.imshow(rgb_default, origin='lower')
    add_scalebar(rgb, scalebar_t, label='5 kpc', color='white', size_vertical=0.2, fontproperties=fm.FontProperties(weight='bold', size=15))
    plt.text(0.04, 0.92, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='left', verticalalignment='center',
                transform = rgb.transAxes, fontsize=25, color='white')
    #plt.text(0.8, 0.05, '%.1f\" = %.2f kpc' % (theta.value, distance_kpc.value),  horizontalalignment='center', verticalalignment='center', 
    #            transform = rgb.transAxes, fontsize=25, color='white', fontweight='bold')
    #plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
    lon = rgb.coords[0]
    lat = rgb.coords[1]
    
    lon.set_ticks_visible(False)
    lon.set_ticklabel_visible(False)
    lat.set_ticks_visible(False)
    lat.set_ticklabel_visible(False)
    lon.set_axislabel('')
    lat.set_axislabel('')
    name_out = 'z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')